In [2]:
import os
from utils.Preprocessing import filter,features,plots
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

This code uploads all files from a country

In [38]:
# file_path = "Organized Data/china/"
# file_list = os.listdir(file_path)
# print(file_list)
# df = pd.DataFrame()

# for file in file_list:
#     # if 'abb_china' in file and file.endswith(".csv"):
#     print(file)
#     df_temp = pd.read_csv(os.path.join(file_path, file), index_col=0)
#     df = pd.concat([df, df_temp])

# # dataset=dataset.sample(frac=1).reset_index(drop=True)
# df.fillna(0, inplace=True)
# df

This code is used to upload data from a single file

In [51]:
df=pd.read_csv('Organized Data/china/abb_china_36501_statistics.csv')
# Fill NaN values with zero
df.fillna(0, inplace=True)
df

,Unnamed: 0,TFF,CT,TAV,SHOT_COUNT,STATISTICS_ID,MOLD_ID,MOLD_CODE,COUNTER_ID,CDATA_ID,SC,WACT,ACT,TOOLING_TYPE,TOTAL_CAVITIES,max_shots,DATE,HOUR,LIFE_CYCLE,PHASE
0,0,20200922145959,0.0,270,7,40120,36501,2TAP200004R0027,NCM2029I01017,40118,7,110.0,110,Injection Mold,8,1500000,20200922,2020092214,0.000005,1
1,1,20200922160002,0.0,270,0,40285,36501,2TAP200004R0027,NCM2029I01017,40283,7,110.0,110,Injection Mold,8,1500000,20200922,2020092216,0.000005,1
2,2,20200922170004,0.0,272,0,40447,36501,2TAP200004R0027,NCM2029I01017,40445,7,110.0,110,Injection Mold,8,1500000,20200922,2020092217,0.000005,1
3,3,20200922180006,0.0,270,0,40610,36501,2TAP200004R0027,NCM2029I01017,40608,7,110.0,110,Injection Mold,8,1500000,20200922,2020092218,0.000005,1
4,4,20200922190009,0.0,270,0,40772,36501,2TAP200004R0027,NCM2029I01017,40770,7,110.0,110,Injection Mold,8,1500000,20200922,2020092219,0.000005,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16546,16546,20230821070748,0.0,279,0,3163735,36501,2TAP200004R0027,NCM2029I01017,3163726,2181770,110.0,110,Injection Mold,8,1500000,20230821,2023082107,1.454513,4
16547,16547,20230821080749,0.0,278,0,3163748,36501,2TAP200004R0027,NCM2029I01017,3163738,2181770,110.0,110,Injection Mold,8,1500000,20230821,2023082108,1.454513,4
16548,16548,20230821090750,0.0,275,0,3163759,36501,2TAP200004R0027,NCM2029I01017,3163751,2181770,110.0,110,Injection Mold,8,1500000,20230821,2023082109,1.454513,4
16549,16549,20230821100751,0.0,273,0,3163768,36501,2TAP200004R0027,NCM2029I01017,3163764,2181770,110.0,110,Injection Mold,8,1500000,20230821,2023082110,1.454513,4


In [52]:
df=df[['HOUR','CT', 'TAV', 'SHOT_COUNT','PHASE']]
# Assuming df is your DataFrame and 'HOUR' is your time column
df['HOUR'] = pd.to_datetime(df['HOUR'], format='%Y%m%d%H')
df = df.sort_values('HOUR')
df

,HOUR,CT,TAV,SHOT_COUNT,PHASE
0,2020-09-22 14:00:00,0.0,270,7,1
1,2020-09-22 16:00:00,0.0,270,0,1
2,2020-09-22 17:00:00,0.0,272,0,1
3,2020-09-22 18:00:00,0.0,270,0,1
4,2020-09-22 19:00:00,0.0,270,0,1
...,...,...,...,...,...
16546,2023-08-21 07:00:00,0.0,279,0,4
16547,2023-08-21 08:00:00,0.0,278,0,4
16548,2023-08-21 09:00:00,0.0,275,0,4
16549,2023-08-21 10:00:00,0.0,273,0,4


In [55]:
# Selecting features
features_considered = ['CT', 'TAV', 'SHOT_COUNT']

# One-hot encoding the 'PHASE' feature
y = pd.get_dummies(df['PHASE']).values

# Creating the sliding window
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y[i:i + time_steps])
    return np.array(Xs), np.array(ys)


time_steps = 720  # Represents 30 days of hourly data

# reshape to [samples, time_steps, n_features]
X, y = create_dataset(df[features_considered], y, time_steps)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set shape: {X_train.shape}, {y_train.shape}")
print(f"Testing set shape: {X_test.shape}, {y_test.shape}")

Training set shape: (12664, 720, 3), (12664, 720, 4)
Testing set shape: (3167, 720, 3), (3167, 720, 4)


In [60]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.layers import TimeDistributed

# Assuming X_train, y_train, X_test, y_test are your training and testing sets
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[2]

# Function to create the model
def create_model(optimizer='adam'):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(n_timesteps,n_features)))
    model.add(TimeDistributed(Dense(n_outputs, activation='softmax')))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model
# # Function to fine-tune hyperparameters
# def fine_tune_model(X_train, y_train):
#     model = KerasClassifier(build_fn=create_model, verbose=0)
#     # define the grid search parameters
#     optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
#     param_grid = dict(optimizer=optimizer)
#     grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
#     grid_result = grid.fit(X_train, y_train)
#     # summarize results
#     print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
#     means = grid_result.cv_results_['mean_test_score']
#     stds = grid_result.cv_results_['std_test_score']
#     params = grid_result.cv_results_['params']
#     for mean, stdev, param in zip(means, stds, params):
#         print("%f (%f) with: %r" % (mean, stdev, param))

# Create the LSTM model
model = create_model()

# Train the model
model.fit(X_train, y_train, epochs=30, batch_size=32, verbose=2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {loss}, Accuracy: {accuracy}")

# Fine-tune hyperparameters
# fine_tune_model(X_train, y_train)


Epoch 1/30
396/396 - 124s - loss: 1.1401 - accuracy: 0.4827
Epoch 2/30
396/396 - 117s - loss: 0.9619 - accuracy: 0.6013
Epoch 3/30
396/396 - 127s - loss: 0.8625 - accuracy: 0.6571
Epoch 4/30
396/396 - 131s - loss: 0.7907 - accuracy: 0.6934
Epoch 5/30
396/396 - 137s - loss: 0.7323 - accuracy: 0.7072
Epoch 6/30
396/396 - 138s - loss: 0.6683 - accuracy: 0.7369
Epoch 7/30
396/396 - 137s - loss: 0.6153 - accuracy: 0.7659
Epoch 8/30
396/396 - 122s - loss: 0.5816 - accuracy: 0.7766
Epoch 9/30
396/396 - 125s - loss: 0.5562 - accuracy: 0.7821
Epoch 10/30
396/396 - 157s - loss: 0.5335 - accuracy: 0.7884
Epoch 11/30
396/396 - 141s - loss: 0.5084 - accuracy: 0.7993
Epoch 12/30
396/396 - 139s - loss: 0.4994 - accuracy: 0.8014
Epoch 13/30
396/396 - 140s - loss: 0.4787 - accuracy: 0.8089
Epoch 14/30
396/396 - 145s - loss: 0.4649 - accuracy: 0.8139
Epoch 15/30
396/396 - 135s - loss: 0.4564 - accuracy: 0.8168
Epoch 16/30
396/396 - 134s - loss: 0.4444 - accuracy: 0.8216
Epoch 17/30
396/396 - 139s - loss

In [61]:
model.save('Weights/abb_china_36501.h5')

In [7]:
from keras.models import load_model
model=load_model('Weights/abb_china_36501.h5')

Testing on other dataset

In [8]:
df = pd.read_csv('Organized Data/china/abb_china_36505_statistics.csv')
df.fillna(0, inplace=True)
df=df[['HOUR','CT', 'TAV', 'SHOT_COUNT','PHASE']]
# Assuming df is your DataFrame and 'HOUR' is your time column
df['HOUR'] = pd.to_datetime(df['HOUR'], format='%Y%m%d%H')
df = df.sort_values('HOUR')
features_considered = ['CT', 'TAV', 'SHOT_COUNT']

# One-hot encoding the 'PHASE' feature
y = pd.get_dummies(df['PHASE']).values

# Creating the sliding window
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y[i:i + time_steps])
    return np.array(Xs), np.array(ys)


time_steps = 720  # Represents 30 days of hourly data

# reshape to [samples, time_steps, n_features]
X, y = create_dataset(df[features_considered], y, time_steps)

loss, accuracy = model.evaluate(X, y, verbose=2)
print(f"Loss: {loss}, Accuracy: {accuracy*100}")

457/457 - 23s - loss: 3.7582 - accuracy: 0.3081
Loss: 3.758192300796509, Accuracy: 30.810141563415527


Test for all files in a folder

In [9]:
# Folder containing CSV files
folder_path = 'Organized Data/china/'

# Loop through all CSV files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        # Construct the full path to the CSV file
        file_path = os.path.join(folder_path, filename)

        # Read the CSV file
        df = pd.read_csv(file_path)
        df.fillna(0, inplace=True)
        df = df[['HOUR', 'CT', 'TAV', 'SHOT_COUNT', 'PHASE']]

        # Assuming df is your DataFrame and 'HOUR' is your time column
        df['HOUR'] = pd.to_datetime(df['HOUR'], format='%Y%m%d%H')
        df = df.sort_values('HOUR')
        features_considered = ['CT', 'TAV', 'SHOT_COUNT']

        # One-hot encoding the 'PHASE' feature
        y = pd.get_dummies(df['PHASE']).values

        # Creating the sliding window
        def create_dataset(X, y, time_steps=1):
            Xs, ys = [], []
            for i in range(len(X) - time_steps):
                v = X.iloc[i:(i + time_steps)].values
                Xs.append(v)
                ys.append(y[i:i + time_steps])
            return np.array(Xs), np.array(ys)

        time_steps = 720  # Represents 30 days of hourly data

        # Reshape to [samples, time_steps, n_features]
        X, y = create_dataset(df[features_considered], y, time_steps)

        # Evaluate the model on the data
        loss, accuracy = model.evaluate(X, y, verbose=2)

        # Print filename, loss, and accuracy
        print(f"File: {filename}, Loss: {loss}, Accuracy: {accuracy * 100}")

537/537 - 28s - loss: 4.7337 - accuracy: 0.2501
File: abb_china_36405_statistics.csv, Loss: 4.733672142028809, Accuracy: 25.009414553642273
550/550 - 29s - loss: 5.1852 - accuracy: 0.2005
File: abb_china_36435_statistics.csv, Loss: 5.185242176055908, Accuracy: 20.052753388881683
475/475 - 24s - loss: 3.9225 - accuracy: 0.3269
File: abb_china_36440_statistics.csv, Loss: 3.922511100769043, Accuracy: 32.69103765487671
464/464 - 23s - loss: 3.5564 - accuracy: 0.3281
File: abb_china_36445_statistics.csv, Loss: 3.556405544281006, Accuracy: 32.812339067459106
489/489 - 24s - loss: 3.3576 - accuracy: 0.2827
File: abb_china_36450_statistics.csv, Loss: 3.357578754425049, Accuracy: 28.271248936653137
490/490 - 24s - loss: 6.8716 - accuracy: 0.0965
File: abb_china_36455_statistics.csv, Loss: 6.87162446975708, Accuracy: 9.646565467119217
508/508 - 25s - loss: 3.1375 - accuracy: 0.3624
File: abb_china_36460_statistics.csv, Loss: 3.1375231742858887, Accuracy: 36.24332249164581
549/549 - 27s - loss: 2